# Lecture 5: Convolutional Neural Networks

In this lecture, we will introduce Convolutional Neural Networks (CNN).

CNN architecture is widely used in image recognition tasks. However, it can also be used in other domains such as Natural Language Processing and speech recognition. Let's focus on the application in NLP and reproduce WaveNet.

CNN papers:
- LeNet: [LeCun et al. 1989](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)
- AlexNet: [Krizhevsky et al. 2012](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)
- WaveNet: [van den Oord et al. 2016](https://arxiv.org/pdf/1609.03499)

## Importing libraries

In [ ]:
import os
import matplotlib.pyplot as plt
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from src.utils import load_text, set_seed, configure_device

## Configuration

In [ ]:
@dataclass
class CNNConfig:
    root_dir: str = os.getcwd() + "/../../"
    dataset_path: str = "data/names.txt"
    device: torch.device = torch.device('cpu')  # Automatic device configuration

    # Tokenizer
    vocab_size: int = 0  # Set later

    # Model
    context_size: int = 5

    # Training
    val_size: float = 0.1
    val_interval: int = 1000
    batch_size: int = 32

    seed: int = 101

## Reproducibility

In [ ]:
set_seed(CNNConfig.seed)

## Device

In [ ]:
CNNConfig.device = configure_device()

## Dataset

In [ ]:
# Load text and split by lines
names = load_text(CNNConfig.root_dir + CNNConfig.dataset_path).splitlines()

## Tokenizer

In [ ]:
chars = [chr(i) for i in range(97, 123)]  # all alphabet characters
chars.insert(0, ".")  # Add special token
CNNConfig.vocab_size = len(chars)
str2idx = {char: idx for idx, char in enumerate(chars)}
idx2str = {idx: char for char, idx in str2idx.items()}

## Preprocessing

In [ ]:
# Train-Val Split
train_names, val_names = train_test_split(names, test_size=CNNConfig.val_size, random_state=CNNConfig.seed)

In [ ]:
# Dataset and DataLoader
class NamesDataset(Dataset):
    def __init__(self, _names, context_size):
        self.inputs, self.targets = [], []

        for name in _names:
            context = [0] * context_size

            for char in name + ".":
                idx = str2idx[char]
                self.inputs.append(context)
                self.targets.append(idx)
                context = context[1:] + [idx]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.inputs[idx])
        target_id = torch.tensor(self.targets[idx])
        return input_ids, target_id

train_dataset = NamesDataset(train_names, context_size=CNNConfig.context_size)
val_dataset = NamesDataset(val_names, context_size=CNNConfig.context_size)
train_loader = DataLoader(train_dataset, batch_size=CNNConfig.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CNNConfig.batch_size, shuffle=False)

## Model

### Multi-Layer Perceptron (MLP)

Let's discuss the architecture of a Multi-Layer Perceptron (MLP).

![MLP](../../assets/mlp.png)

Q1: How do the embedding tokens communicate with each other? What operation is performed to do so?

Q2: Let's increase the context size to 16, 64, ... What are the restrictions of increasing the context size?



In [ ]:
################################################################################
# TODO:                                                                        #
# Write your answer to the questions above.                                    #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# A1:
# A2:
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

### Convolutional Neural Network (CNN)

![CNN](../../assets/cnn.png)

Instead of connecting each token to all other tokens, CNN uses convolutional layers to connect tokens within a certain range.
